In [1]:
!pip install -q peft datasets
!pip install -q accelerate bitsandbytes

In [2]:
!pip install -q transformers

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup ,TrainingArguments
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorWithPadding

In [4]:
device = "cuda"
model_name_or_path = "google/flan-t5-large"
tokenizer_name_or_path = "google/flan-t5-large"

text_column = "prompt"
label_column = "label"

max_length = 256

In [5]:
from datasets import load_from_disk

# Specify the directory where the dataset is saved
saved_directory = "/content/drive/MyDrive/Colab Notebooks/Table to insights/Data/Analytical Datset"

# Load the dataset from the specified directory
dataset = load_from_disk(saved_directory)


In [6]:
dataset['train'].features

{'table': [{'cpu_usage': Value(dtype='float64', id=None),
   'date': Value(dtype='string', id=None),
   'diskio_usage': Value(dtype='float64', id=None),
   'ram_usage': Value(dtype='float64', id=None)}],
 'prompt': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

# Trainer API

In [7]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

In [8]:
def preprocess_data(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs,padding="max_length",max_length = max_length,truncation=True, return_tensors="pt")
    labels = tokenizer(targets, padding="max_length", max_length = max_length,truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

In [9]:
processed_datasets = dataset.map(
    preprocess_data,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/516 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/129 [00:00<?, ? examples/s]

In [10]:
processed_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 516
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 129
    })
})

In [11]:
train_dataset = processed_datasets['train']
validation_dataset = processed_datasets['validation']

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path,load_in_8bit=True,device_map='auto')

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
model.train()
model.resize_token_embeddings(len(tokenizer))
def create_peft_config(model):

    peft_config = PrefixTuningConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        inference_mode=False,
        num_virtual_tokens=20
    )

    # prepare int-8 model for training
    # model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

model, peft_config = create_peft_config(model)

trainable params: 983,040 || all params: 784,075,776 || trainable%: 0.12537563716290606


In [14]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/Table to insights/Flan"

In [15]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_hf",
    learning_rate = 1e-4,
    num_train_epochs=1,
    gradient_accumulation_steps= 1,
    per_device_train_batch_size= 1,
    gradient_checkpointing=False
)

# Auto trainer

In [16]:
from transformers import Trainer

In [17]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        # data_collator=default_data_collator,
    )

In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,4.454900
20,4.257800
30,4.124800
40,4.313900
50,4.152900
60,3.987500
70,4.167600
80,4.323600
90,4.227900
100,4.116400


TrainOutput(global_step=516, training_loss=3.850934169089147, metrics={'train_runtime': 268.9015, 'train_samples_per_second': 1.919, 'train_steps_per_second': 1.919, 'total_flos': 594607406579712.0, 'train_loss': 3.850934169089147, 'epoch': 1.0})

In [19]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/Table to insights/Flan/prefix_tuned_model_256")